In [2]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Read the CSV file
df = pd.read_csv('result/sorted.csv')

# Convert start_time and end_time to datetime
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])

# Create the base map
fig = px.scatter_geo(
    df,
    lat='latitude',
    lon='longitude',
    hover_name='station',
    projection='natural earth',
    scope='north america',
    color='mean_snr_db',  # Color points by mean_snr_db
    color_continuous_scale=px.colors.sequential.Viridis,  # Set color scale
    title='Infrasound Events in North America'
)

fig.update_geos(
    showcountries=True,
    countrycolor="Black",
    showsubunits=True,
    subunitcolor="Blue"
)

fig.update_traces(
    marker=dict(size=10, opacity=0.7),
    selector=dict(mode='markers')
)

fig.update_layout(
    geo=dict(
        lataxis_range=[20, 80],
        lonaxis_range=[-170, -50]
    )
)

# Create the Dash app
app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='map-graph', figure=fig),
    html.Div([
        html.Label('Select Station:'),
        dcc.Dropdown(
            id='station-dropdown',
            options=[{'label': i, 'value': i} for i in df['station'].unique()] + [{'label': 'All', 'value': 'All'}],
            value=['All'],
            multi=True
        ),
        html.Label('Select Cluster:'),
        dcc.Dropdown(
            id='cluster-dropdown',
            options=[{'label': i, 'value': i} for i in df['cluster'].unique()] + [{'label': 'All', 'value': 'All'}],
            value=['All'],
            multi=True
        ),
        html.Label('Search Data:'),
        dcc.Input(
            id='search-input',
            type='text',
            placeholder='Search...',
            debounce=True
        )
    ], style={'width': '48%', 'display': 'inline-block'})
])

@app.callback(
    Output('map-graph', 'figure'),
    [Input('station-dropdown', 'value'),
     Input('cluster-dropdown', 'value'),
     Input('search-input', 'value')]
)
def update_graph(selected_stations, selected_clusters, search_value):
    filtered_df = df.copy()
    
    if selected_stations is None or 'All' in selected_stations:
        selected_stations = df['station'].unique()
    if selected_clusters is None or 'All' in selected_clusters:
        selected_clusters = df['cluster'].unique()
    
    filtered_df = filtered_df[
        filtered_df['station'].isin(selected_stations) &
        filtered_df['cluster'].isin(selected_clusters)
    ]

    if search_value:
        search_value = search_value.lower()
        filtered_df = filtered_df[filtered_df.apply(lambda row: row.astype(str).str.contains(search_value, case=False).any(), axis=1)]

    fig = px.scatter_geo(
        filtered_df,
        lat='latitude',
        lon='longitude',
        hover_name='station',
        hover_data=['cluster', 'mean_peak_db', 'max_peak_db', 'mean_duration_sec', 'mean_background_db', 'max_background_db', 'mean_snr_db', 'max_snr_db', 'max_duration_sec', 'start_time', 'end_time'],
        projection='natural earth',
        scope='north america',
        color='mean_snr_db',  # Use mean_snr_db for color
        color_continuous_scale=px.colors.sequential.Viridis  # Set color scale
    )
    
    fig.update_geos(
        showcountries=True,
        countrycolor="Black",
        showsubunits=True,
        subunitcolor="Blue"
    )
    
    fig.update_traces(
        marker=dict(size=10, opacity=0.7),
        selector=dict(mode='markers')
    )
    
    fig.update_layout(
        title='Infrasound Events in North America',
        geo=dict(
            lataxis_range=[20, 80],
            lonaxis_range=[-170, -50]
        )
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=2233)
